# Analysis: calculation of the FuncUse and his ratio
  
For all patients and all months, calcule the FuncUse of the paretic and non-paretic arm and his ratio for each day.  

This script is inspired by Victor Fernando Lopes De Souza's script.

## 1. Loading extensions and necessary libraries

These cells load the `autoreload` extension to automatically reload all modules before executing the Python code, and imports the necessary libraries for data processing and graphical visualization.

In [2]:
# Load the autoreload extension
%load_ext autoreload

In [ ]:
# Set IPython magic extension to autoreload all modules every time before executing the Python code
%autoreload 2

# Standard library imports
import sys
import os
import re

# Install required libraries
# If some of these libraries are already installed and you don't want to install 
# them again, please comment the concerned lines in this block

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install IPython

# Third party imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.signal import butter, filtfilt

# matplotlib config
mpl.rcParams['agg.path.chunksize'] = 10000

# Configure working directory and source paths
notebook_dir = os.getcwd()
sys.path.append(os.path.join(notebook_dir,"..", "sources"))

import handle_data, functional_metrics

## 2. Configure working directory and source paths

This block ensures that the working directory is set to the project root and that the `sources` folder is added to the Python path to allow proper imports. It also defines the path to the actimetry data folder.

In [4]:
# If the present working directory is the directory of this file, change the path to the project root
import os
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Path to actimetry data
act_folder = "data/data_actimetry/"

## 3. Define months you want to process
`months` is a list of integers representing the months to analyze, from 1 (first month) to 6 (sixth month).

In [5]:
# Months to analyse
months = [1]

## 4. Preprocessing parameters
This section defines the signal preprocessing parameters used for analyzing accelerometer data. It includes settings for resampling frequency, low-pass filtering, and the duration of time windows used for feature extraction.

In [ ]:
# Resampling
resampling_freq = 50 # Hz

# Low pass filter
filter_butter_cutoff = 1 # Hz

# Windowing
seconds_per_window = 2 # s 

## 5. View patients metadata
`participants_X.csv` :   
- `X` is the ID of the month of record (from `1` = first month to `6` = sixth month)
- The file contains, for each patient: 
    - `folder_name` : Patient ID (e.g., C1P30 corresponding to investigation center 1 and the 30th patient included in the study)
    - `is_patient` : Patient (True) or healthy subject (False)
    - `parent_folder` : Actimetric data folder name
    - `paretic_side` : Right or left paretic side
    - `start_day`, `start_month`, `start_year` : Start date of recording
    - `end_day`, `end_month`, `end_year` : End date of recording
    - `age` : Subject's age on the day of inclusion
    - `FMScore` : The Fugl-Meyer score (upper-limb function, /66)
    - `freq` : Sampling frequency
    - `time_stroke` : Number of months post-stroke
    - `laterality` : Laterality
    - `barthel` : Barthel score (autonomy index, /100)
    - `bbt_paretic` and `bbt_non_paretic` : Box and Block test scores (gross motor function)

In [8]:
# Define the metadata file path
participants_csv_path = "data/participants_1.csv"

# Check if the CSV file exists
if not os.path.exists(participants_csv_path):
    raise FileNotFoundError(f"The file {participants_csv_path} was not found.")

# Load participants metadata
participants_info = pd.read_csv(participants_csv_path, sep=';')

# Display sample of metadata
print("\nTable 1. Patients metadata")
participants_info.head()


Table 1. Patients metadata


,folder_name,is_patient,parent_folder,paretic_side,start_day,end_day,start_month,end_month,start_year,end_year,age,FMScore,freq,time_stroke,laterality,barthel,bbt_paretic,bbt_non_paretic
0,C1P01_M1,True,data_actimetry,left,21,29,3,3,2025,2025,23,33,50,121,right,100,0,64
1,C1P30_M1,True,data_actimetry,right,29,7,11,12,2024,2024,67,45,50,36,right,95,17,46
2,C1P31_M1,True,data_actimetry,left,10,18,1,1,2025,2025,75,49,50,123,right,80,35,52
3,C1P32_M1,True,data_actimetry,right,29,7,11,12,2024,2024,78,45,50,121,right,90,20,50
4,C1P33_M1,True,data_actimetry,left,29,7,11,12,2024,2024,81,54,50,33,right,85,18,38


## 6. Calculation of the FuncUseRatioPerDay for ALL patients and ALL months

Make an overall report of the daily FuncUse of the paretic arm, the non-paretic arm, and the daily FuncUseRatio for each patient and each month.  
A FuncUse is a functional movement of the upper limb that satisfies two conditions: the arm elevation angle must be greater than 30°, and the movement must occur within a range of -30° to +30° around the horizontal plane [Leuenberger, 2017].

In [ ]:
def get_functional_uses_per_day(folder_name: str, month: str) -> pd.DataFrame:
    """
    Get functional use counts per day for the given patient and month.

    This function processes accelerometer data for a specific patient and month to calculate the 
    number of functional uses per day. It extracts and processes the accelerometer data, calculates 
    the functional movement angles, and checks if those movements are functional. Finally, it returns 
    a DataFrame with the functional use counts for each day of the specified month.

    Parameters
    -----------
    folder_name : str
        The name of the folder containing the patient's data. The patient's ID is extracted from this name.

    month : list of str
        A list of months for which the functional use data is to be analyzed.

    Returns
    --------
    functional_uses_per_day_df : pandas.DataFrame
        A DataFrame where each row represents a day of the month. The columns include:
        - 'ID' : The patient's ID.
        - 'month' : The month of analysis in the range of 1 (first month) to 6 (sixth month).
        - 'day' : The day of the month in the range of 1 (first day) to 7 (seventh day).
        - 'FuncUse_non_paretic_day' : The count of functional uses for the non-paretic arm.
        - 'FuncUse_paretic_day' : The count of functional uses for the paretic arm.
        - 'FuncUseRatio_day' : The ratio of functional use on the paretic arm to the total functional uses (paretic + non-paretic).
    """
    try:
        # Get data
        time_index, acceleration_xyzn, is_patient, FM = handle_data.extract_data(folder_name, filter_butter_cutoff=filter_butter_cutoff, resampling_freq=resampling_freq, month=month)
        
        # Convert time index to seconds
        time_index_values = (time_index.values - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')
        
        # Partition data
        windows_small, time_indexes_small = handle_data.partition(acceleration_xyzn, time_index_values, seconds_per_window=seconds_per_window)

        # Get angles
        alphas = functional_metrics.get_angle_from_horizontal(windows_small[:, :, :, 1], windows_small[:, :, :, 3])

        # Get if window contains a functional movement for different (symmetric) angle thresholds (30)
        angle_thresholds = [30]
        is_functional = {i: functional_metrics.test_functional(alphas=alphas, threshold_symregion_degrees=i, threshold_amp_degrees=30) for i in angle_thresholds}
        
        # Get functional count per day
        functional_count_per_day = functional_metrics.get_functional_count_per_day(is_functional[30], time_indexes_small)
        functional_uses_per_day_df = pd.DataFrame(functional_count_per_day)

        # Add patient info to the dataframe
        ID = folder_name.split('_')[0]
        functional_uses_per_day_df.insert(0, 'ID', ID)
        functional_uses_per_day_df.insert(1, 'month', month)
        functional_uses_per_day_df.insert(2, 'day', range(1, functional_uses_per_day_df.shape[0] + 1))
        functional_uses_per_day_df.columns = ['ID', 'month', 'day', 'FuncUse_non_paretic_day', 'FuncUse_paretic_day']
        functional_uses_per_day_df['FuncUseRatio_day'] = functional_uses_per_day_df['FuncUse_paretic_day'] / (functional_uses_per_day_df['FuncUse_paretic_day'] + functional_uses_per_day_df['FuncUse_non_paretic_day'])

        return functional_uses_per_day_df

    except Exception as e:
        print(f"Error processing functional uses per day for {folder_name} in month {month}: {e}")
        return None

print("\nCOUNTING OF FUNCTIONAL MOVEMENTS AND CALCULATION OF ITS RATIO FOR EACH DATA FILE:")

# Initialize an empty list to store all daily results
all_daily_results = []

# Loop over each month
for month in months:
    # Load participants info for the current month
    participants_info = pd.read_csv(f'data/participants_{month}.csv', sep=';')
    
    # Process each participant in the current month
    for _, row in participants_info.iterrows():
        folder_name = row['folder_name']
        print(f'-----\nProcessing {folder_name} for month {month}')
        try:
            # Get the functional uses per day for this folder_name and month
            daily_result = get_functional_uses_per_day(folder_name, month)
            
            # If result is not None, append it to the global list
            if daily_result is not None:
                all_daily_results.append(daily_result)
        
        except Exception as e:
            print(f'Error processing {folder_name}: {e}')

# Concatenate all daily results into a single DataFrame
all_daily_results_df = pd.concat(all_daily_results, axis=0)

# Sort the dataframe by patient (ID), then by month, and finally by day
all_daily_results_df = all_daily_results_df.sort_values(by=['ID', 'month', 'day']).reset_index(drop=True)

# Save the final daily results to a CSV file
all_daily_results_df.to_csv('results/results_FuncUsePerDay_all_patients.csv', index=False)

print("-----\nProcessing complete.\nResults saved to 'results_FuncUsePerDay_all_patients.csv' in 'results' folder.")


COUNTING OF FUNCTIONAL MOVEMENTS AND CALCULATION OF ITS RATIO FOR EACH DATA FILE:
-----
Processing C1P01_M1 for month 1
-----
Processing C1P30_M1 for month 1
Error processing functional uses per day for C1P30_M1 in month 1: [Errno 2] No such file or directory: 'data/data_actimetry/C1P30_M1/right.csv'
-----
Processing C1P31_M1 for month 1
Error processing functional uses per day for C1P31_M1 in month 1: [Errno 2] No such file or directory: 'data/data_actimetry/C1P31_M1/left.csv'
-----
Processing C1P32_M1 for month 1
Error processing functional uses per day for C1P32_M1 in month 1: [Errno 2] No such file or directory: 'data/data_actimetry/C1P32_M1/right.csv'
-----
Processing C1P33_M1 for month 1
Error processing functional uses per day for C1P33_M1 in month 1: [Errno 2] No such file or directory: 'data/data_actimetry/C1P33_M1/left.csv'
-----
Processing complete.
Results saved to 'results_FuncUsePerDay_all_patients.csv' in 'results' folder.


## 7. Keep only the complete days of wearing
Incomplete wearing days can lead to aberrant FuncUseRatio values.  
<100 FuncUse is the threshold defined to identify the days when the accelerometers were not worn enough.

In [ ]:
# Remove rows where the value in the non-paretic FuncUse is less than 100
all_daily_results_df_filtered = all_daily_results_df[all_daily_results_df['FuncUse_non_paretic_day'] >= 100]

# Save the filtered DataFrame to a new CSV file (or overwrite the existing file if necessary)
all_daily_results_df_filtered.to_csv('results/results_FuncUsePerDay_all_patients_filtered.csv', index=False)

# Print the filtered table
print("\nTable 2. Daily functional use count and ratio")
display(all_daily_results_df_filtered)


Table 2. Daily functional use count and ratio


,ID,month,day,FuncUse_non_paretic_day,FuncUse_paretic_day,FuncUseRatio_day
0,C1P01,1,1,654.0,38.0,0.054913
1,C1P01,1,2,553.0,22.0,0.038261
2,C1P01,1,3,674.0,41.0,0.057343
3,C1P01,1,4,632.0,42.0,0.062315
4,C1P01,1,5,608.0,21.0,0.033386
5,C1P01,1,6,517.0,31.0,0.056569
6,C1P01,1,7,689.0,23.0,0.032303
7,C1P01,1,8,153.0,10.0,0.061350
